In [1]:
#import kaggle as kg
import pandas as pd
import numpy as np
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as s
import pickle
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

In [38]:
os.environ["USERNAME"] = "aimlrl"
os.environ["KEY"] = "f9fcb3d12c2c4845724a50f0cdeab5ea"

In [3]:
#kg.api.authenticate()

In [ ]:
"""kg.api.dataset_download_files(dataset="ehtishamsadiq/uncleaned-laptop-price-dataset",
                              path="./dataset",unzip=True)"""

In [2]:
data = pd.read_csv("project_root/dataset/laptopData.csv")

In [ ]:
data.head()

In [ ]:
data.isna().sum()

In [5]:
data.dropna(axis=0,inplace=True)

In [6]:
data.drop(labels=data.columns[0],axis=1,inplace=True)

In [7]:
CATEGORICAL_COLUMNS = list(data.columns[0:2]) + list(data.columns[3:5]) + list(data.columns[6:9])
NUMERIC_COLUMNS = list(set(data.columns).difference(set(CATEGORICAL_COLUMNS)))

In [ ]:
CATEGORICAL_COLUMNS

In [ ]:
NUMERIC_COLUMNS

In [10]:
def label_encode_column(column_name):

    data[column_name].unique()

    d = dict()
    for value in data[column_name].unique():
        d[value] = data[data[column_name] == value][data.columns[-1]].mean()

    input_target_df = pd.DataFrame(data={column_name:d.keys(),"Mean Price":d.values()})
    input_target_df.sort_values(by="Mean Price",inplace=True)
    input_target_df.reset_index(inplace=True,drop=True)

    data[column_name] = data[column_name].replace(to_replace=list(input_target_df[column_name]),
                              value=list(input_target_df.index)).infer_objects(copy=False)
    
    return dict(zip(list(input_target_df[column_name]),list(input_target_df.index)))

In [11]:
categorical_columns = list(data.columns[0:2]) + list(data.columns[3:5]) + list(data.columns[6:9])

In [ ]:
columns_label_encodings = dict()

for column_name in categorical_columns:

    label_encoding_dict = label_encode_column(column_name)
    columns_label_encodings[column_name] = label_encoding_dict

In [10]:
with open("columns_label_encodings.pkl","wb") as file_handle:
    pickle.dump(columns_label_encodings,file_handle)

In [13]:
data["Ram"] = data["Ram"].apply(lambda x: float(x.split("GB")[0]))

In [ ]:
plt.hist(data["Weight"])

In [ ]:
data["Weight"].replace(to_replace="?",value=data["Weight"].value_counts().index[0],
                       inplace=True)

In [16]:
data["Weight"] = data["Weight"].apply(lambda x: float(x.split("kg")[0]))

In [ ]:
data["Inches"].replace(to_replace="?",value=data["Inches"].value_counts().index[0],
                       inplace=True)

In [18]:
data["Inches"] = data["Inches"].astype("float64")

In [19]:
data.drop_duplicates(inplace=True,ignore_index=True)

In [ ]:
data_corr = data.corr()
upper_traingle_mask = np.triu(np.ones_like(data_corr,dtype=bool))
sns.heatmap(data=data_corr,center=0,annot=True,square=True,xticklabels=True,yticklabels=True,mask=upper_traingle_mask)
plt.show()

In [21]:
selected_features = list(data_corr[data_corr[data.columns[-1]] > 0.5].index)

In [22]:
refined_data = data[selected_features]

In [ ]:
refined_data

In [24]:
X_transpose = np.array(refined_data.iloc[:,0:6])
y = np.array(refined_data.iloc[:,-1]).reshape(refined_data.shape[0],1)

In [25]:
def engineered_feaures(X_transpose,features_degree,is_only_interaction):

    nth_degree_feature_engineer = PolynomialFeatures(degree=features_degree,interaction_only=is_only_interaction)
    X_transpose_engineered = nth_degree_feature_engineer.fit_transform(X_transpose)

    return X_transpose_engineered

In [26]:
def normalize_data(X_transpose):

    zero_mean_one_std_scaler = StandardScaler()
    X_bar_transpose = zero_mean_one_std_scaler.fit_transform(X_transpose)

    return X_bar_transpose

In [27]:
def train_cv_test_split(X_transpose, y, train_frac, cv_frac):

    X_train_transpose = X_transpose[0:int(train_frac*X_transpose.shape[0])]
    y_train = y[0:int(train_frac*y.shape[0])]

    X_cv_transpose = X_transpose[int(train_frac*X_transpose.shape[0]):int((train_frac+cv_frac)*X_transpose.shape[0])]
    y_cv = y[int(train_frac*y.shape[0]):int(train_frac+cv_frac*y.shape[0])]

    X_test_transpose = X_transpose[int((train_frac+cv_frac)*X_transpose.shape[0]):]
    y_test = y[int((train_frac+cv_frac)*y.shape[0]):]

    return (X_train_transpose,y_train), (X_cv_transpose,y_cv), (X_test_transpose,y_test)

In [ ]:
plt.xlabel("Price of a Used Laptop")
plt.ylabel("Frequency")
plt.hist(y)

In [ ]:
plt.xlabel("Price of a Used Laptop")
sns.distplot(y,fit=s.norm)

In [ ]:
plt.xlabel("Log Transformed Price of a Used Laptop")
plt.ylabel("Frequency")
plt.hist(np.log(y))

In [ ]:
plt.xlabel("Log Transformed Price of a Used Laptop")
sns.distplot(np.log(y),fit=s.norm)

In [32]:
X_transpose_engineered = engineered_feaures(X_transpose,2,False)
X_bar_transpose = normalize_data(X_transpose_engineered)
(X_train_transpose,y_train), (X_cv_transpose,y_cv), (X_test_transpose,y_test) = train_cv_test_split(X_bar_transpose,y,0.7,0.2)

In [33]:
def mse(theta0,theta):

    return np.mean((y_train - (theta0 + np.matmul(X_train_transpose,theta)))**2)

In [34]:
def del_mse_by_del_theta(theta0,theta):

    error_transpose = np.transpose((theta0 + np.matmul(X_train_transpose,theta) - y_train))

    del_by_del_theta0 = np.mean(error_transpose)
    del_by_del_theta = (1/y_train.shape[0])*np.transpose(np.matmul(error_transpose,X_train_transpose))

    return [del_by_del_theta0,del_by_del_theta]

In [35]:
def training(epsilon=10**(-4),epoch_counter=0,theta0_initial=0,
             theta_initial = np.zeros((X_train_transpose.shape[1],1)),tol=10**(-5)):

    while True:

        initial_gradients = del_mse_by_del_theta(theta0_initial,theta_initial)

        theta0_final = theta0_initial - (epsilon * initial_gradients[0])
        theta_final = theta_initial - (epsilon * initial_gradients[1])

        mse_initial_value = mse(theta0_initial,theta_initial)
        mse_final_value = mse(theta0_final,theta_final)

        if abs(mse_initial_value - mse_final_value) < tol:
            break

        epoch_counter += 1

        theta0_initial = theta0_final
        theta_initial = theta_final

        print("Epoch # {}, MSE Value = {}".format(epoch_counter,mse_initial_value))

    return [theta0_final,theta_final]